# Notes

This is intended as a short tutorial on Fight detection, and plotting windowed distributions. <br>


# Imports

In [ ]:
import numpy as np
np.set_printoptions(suppress=True)
np.seterr(divide='ignore', invalid='ignore')
import h5py
import time
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial import distance
import os
import sys
%matplotlib widget
%load_ext autoreload
%autoreload 2

from FightDetector import FightDetector
from WindowedDistributionPlotter import compute_dpp_tet_tet_distribution_plot_with_fight_bouts

# filepaths

In [ ]:
# -------------------------------------------#
# load data and params needed for clustering
# -------------------------------------------#

# the path for data for the fight detector
clusterable_data_loadpath =  './clusterable_data.h5'

# the path to some cluster labels for the clusterable_data
clusterable_data_labels_path = './clusterable_data_labels.h5'

# the path to the parameters used for the clustering
cluster_params_path = './cluster_params.h5'


# -------------------------------------------#
# load tracking results to play with
# -------------------------------------------#
data_loadpath = './FishTank20200130_153857_tracking_results.h5'
with h5py.File(data_loadpath, 'r') as hf:
    traj = hf['tracks_3D_smooth'][:]
    
expName = 'FishTank20200130_153857'

# Create a fight detector

In [ ]:
# --- load the stuff we need to instantiate the fight detector  ---- #

with h5py.File(clusterable_data_loadpath, 'r') as hf:
    expidx_and_decimated_wins_clusterable_data = hf['expidx_and_decimated_wins_clusterable_data'][:]
    prob_vectors_clusterable_data = hf['prob_vectors_clusterable_data'][:]

    
with h5py.File(clusterable_data_labels_path, 'r') as hf:
    cluster_labels = hf['cluster_labels'][:]
    fight_label_number = hf['fight_label_number'][()]
    
    
# load the clustering params dictionary
cluster_params = {}
with h5py.File(cluster_params_path, 'r') as hf:
    for param_key in hf.keys():
        try:
            param_value = hf[param_key][:]
        except:
            param_value = hf[param_key][()]
        cluster_params.update({param_key:param_value})
        

In [ ]:
# ---- make a fight detector ---- #
fd = FightDetector(cluster_params['dpp_bins'], cluster_params['g1_bins'], cluster_params['g2_bins'], 
                   cluster_params['window_size'], cluster_params['window_step'], cluster_params['skip_size'],
                   prob_vectors_clusterable_data, cluster_labels, fight_label_number, 
                   refining_skip_size=2, merge_gap_size_in_windows=5, min_region_window_size=5)

# Detect fight-bouts in the experiment

In [ ]:
t0 = time.perf_counter()

fight_bouts_info = fd.detect_fight_bouts(traj)

tE = time.perf_counter()
print(tE-t0)

In [ ]:
fight_bouts_info

# Plot the windowed distributions with fight-bouts

In [ ]:
t0 = time.perf_counter()

# These variables you can read from the tracking results for paper data.
# For new experiments, you don't know these yet.
# If you don't know, choose winIdx=0, losIdx=1, and set has_clear_winner=False
winIdx = 0
losIdx = 1
has_clear_winner = False

# the fight information is simply what we found above
exp_fight_bout_info = fight_bouts_info

# the trajectory, traj, is also defined above

# plotting params
# expName is set towards the top of this file
figsavepath = f'./{expName}.png'
window_size=6000                            # the window width [frames]
window_step=100                             # the step forward in time between windows [frames]
dpp_bins = np.linspace(0, 30, 300)          # visualization bins
tet_bins = np.linspace(-np.pi, np.pi, 300)  # visualization bins
dpp_vmax=150                                # the max count color threshold when plotting.
tet_vmax=150                                # the max count color threshold when plotting.
num_xticks=10                               # the number of xaxis time ticks
cmap='Blues'                                # colorscheme
fps=100                                     # for frame-minutes conversions
use_cbar=False                              # use/not-use the colorbar of the plots



compute_dpp_tet_tet_distribution_plot_with_fight_bouts(traj, expName, exp_fight_bout_info, figsavepath, dpp_bins, tet_bins,
                                                       winnerIdx=winIdx, loserIdx=losIdx, has_clear_winner=has_clear_winner, 
                                                       window_size=window_size, window_step=window_step, dpp_vmax=dpp_vmax, 
                                                       tet_vmax=tet_vmax, num_xticks=num_xticks, cmap='Blues', fps=fps, use_cbar=use_cbar)


tE = time.perf_counter()
print()
print('finished {0}: {1} s'.format(expName, tE-t0))